<a href="https://colab.research.google.com/github/mftnakrsu/reinforcement-learning/blob/main/deep_q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import gym
import numpy as np
from collections import deque
import keras
from keras.models import Sequential
from keras.layers import Dense
import  random
from colabgymrender.recorder import  Recorder
import tensorflow as tf
import time

In [ ]:
class DQLAgent:

  
  def __init__(self,env):
    #hyperparameter
    self.state_size  =env.observation_space.shape[0]
    self.action_size = env.action_space.n

    self.gamma  = 0.95
    self.l_rate = 0.001

    self.epsilon = 1 # baslangıcta her yeri ara
    self.epsilon_decay=0.995
    self.epsilon_min = 0.01

    self.memory=deque(maxlen=1000)

    self.opt= tf.keras.optimizers.Adam(lr=self.l_rate)
    self.model= self.build_model() # neural network q learning de kullanılan
    

  def build_model(self):
    """
    neural network forl DQL
    """
    model = Sequential()
    model.add(Dense(48,input_dim=self.state_size,activation="tanh"))
    model.add(Dense(self.action_size,activation="linear"))
    model.compile(loss="mse",optimizer=self.opt)
    return model


  def remember(self,state,action,reward,next_state,done):
    """
    storage(state action reward done)
    """
    self.memory.append((state,action,reward,next_state,done))


  def act(self,state):
    """
    acting
    """
    if random.uniform(0,1)<=self.epsilon:
      return env.action_space.sample()
    else:
      act_values = self.model.predict(state)
      return np.argmax(act_values[0])


  def replay(self,batch_size):
    # training
    if len(self.memory) < batch_size:
      return
    minibatch = random.sample(self.memory,batch_size)

    for (state,action,reward,next_state,done) in minibatch:
      if done:
        target = reward
      else:
        target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])

      train_target = self.model.predict(state)
      train_target[0][action] = target

      self.model.fit(state, train_target,verbose=0)


  def adaptiveEGreedy(self):
    if self.epsilon > self.epsilon_min:
      self.epsilon *= self.epsilon_decay
  

if __name__=="__main__":
  # initilaize env and agent
  
  episode=100
  batch_size=16
  env = gym.make('CartPole-v0')
  agent = DQLAgent(env)

  for e in range(episode):
    # init env
    state = env.reset()
    state=np.reshape(state,[1,4])
    time = 0
    while True:
      #act
      action = agent.act(state)
      #step
      next_state,reward,done,_ = env.step(action)
      next_state=np.reshape(next_state,[1,4])

      #remember
      agent.remember(state,action,reward,next_state,done)
      #update stage
      state = next_state
      #replay
      agent.replay(batch_size)
      #adjust epsilon
      agent.adaptiveEGreedy()

      time+=1

      if done:
        print("Episode: {}, time: {}".format(e,time))
        break

In [ ]:
trained_model=agent
state = env.reset()
state = np.reshape(state,[1,4])
time_t=0

while True:
  env.render()
  trained_model.act(state)
  next_state, reward, done, _ = env.step(action)
  next_state = np.reshape(next_state,[1,4])
  state = next_state
  time_t +=1
  print(time)
  time.sleep(0.4)
  if done:
    break

print("Done !")